# HEALPix Demo: Spatially Varying Photometric Errors

This notebook demonstrates how to use the CustomErrorModel with spatially varying observing conditions defined on HEALPix maps. We'll create realistic sky maps for PSF, airmass, sky brightness, and exposure time, then show how these affect photometric errors across the sky.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
try:
    import healpy as hp
except ImportError:
    print("Installing healpy...")
    !pip install healpy
    import healpy as hp

import sys
sys.path.append('..')
from photerr.custom import CustomErrorModel

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully!")

## 1. Create HEALPix Maps for Observing Conditions

First, we'll create realistic maps of observing conditions that vary across the sky:

In [ ]:
def create_observing_condition_maps(nside=64):
    """Create realistic HEALPix maps for observing conditions."""
    npix = hp.nside2npix(nside)
    
    # Get pixel coordinates
    theta, phi = hp.pix2ang(nside, range(npix))
    dec = np.pi/2 - theta  # Convert to declination
    
    # Get galactic coordinates for realistic sky brightness
    gal_theta, gal_phi = hp.Rotator(coord=['C', 'G'])(theta, phi)
    gal_lat = np.pi/2 - gal_theta
    
    maps = {}
    
    # 1. PSF Maps: Worse seeing at low declination (higher airmass)
    # and some random variation for weather
    base_seeing = 0.8 + 0.4 * np.abs(np.sin(dec))**2  # 0.8" to 1.2"
    weather_noise = 0.2 * np.random.normal(0, 1, npix)
    
    for band in ['g', 'r', 'i', 'z']:
        # Wavelength-dependent seeing (better at longer wavelengths)
        wavelength_factor = {'g': 1.0, 'r': 0.95, 'i': 0.9, 'z': 0.85}[band]
        maps[f'{band}_psf'] = np.clip(
            wavelength_factor * (base_seeing + weather_noise), 0.6, 2.0
        )
    
    # 2. Airmass Maps: Function of declination with some pointing variation
    base_airmass = 1.0 / np.maximum(np.sin(dec + np.pi/4), 0.1)
    pointing_variation = 0.2 * np.random.normal(0, 1, npix)
    
    for band in ['g', 'r', 'i', 'z']:
        maps[f'{band}_airmass'] = np.clip(base_airmass + pointing_variation, 1.0, 3.0)
    
    # 3. Sky Brightness: Varies with galactic latitude and ecliptic latitude
    # Darker at high galactic latitude, brighter near galactic plane
    gal_plane_brightness = 2.0 * np.exp(-np.abs(gal_lat) / 0.3)
    
    # Zodiacal light (simplified)
    ecl_theta, ecl_phi = hp.Rotator(coord=['C', 'E'])(theta, phi)
    ecl_lat = np.pi/2 - ecl_theta
    zodiacal_light = 0.5 * np.exp(-np.abs(ecl_lat) / 0.5)
    
    # Random moon/airglow variation
    random_brightness = 0.3 * np.random.normal(0, 1, npix)
    
    # Base sky brightness by band (mag/arcsec²)
    base_sky = {'g': 22.0, 'r': 21.0, 'i': 20.0, 'z': 19.0}
    
    for band in ['g', 'r', 'i', 'z']:
        maps[f'{band}_msky'] = base_sky[band] - gal_plane_brightness - zodiacal_light + random_brightness
    
    # 4. Exposure Time: Survey strategy - more time on interesting regions
    # High galactic latitude gets more time (less crowded)
    priority_factor = 1.0 + 0.5 * np.abs(np.sin(gal_lat))
    
    # Some random variation for scheduling
    scheduling_noise = 0.3 * np.random.normal(0, 1, npix)
    
    for band in ['g', 'r', 'i', 'z']:
        base_time = {'g': 60, 'r': 60, 'i': 60, 'z': 60}[band]
        maps[f'{band}_tvis'] = np.clip(
            base_time * priority_factor * (1 + scheduling_noise), 30, 180
        )
    
    return maps, nside

# Create the maps
print("Creating observing condition maps...")
maps, nside = create_observing_condition_maps(nside=64)
print(f"Created maps with nside={nside} ({hp.nside2npix(nside)} pixels)")

## 2. Visualize the Input Maps

Let's visualize the observing condition maps we just created:

In [ ]:
# Plot PSF maps
fig = plt.figure(figsize=(16, 12))
for i, band in enumerate(['g', 'r', 'i', 'z']):
    plt.subplot(2, 2, i+1)
    hp.mollview(maps[f'{band}_psf'], title=f'{band.upper()}-band PSF (arcsec)', 
                sub=(2, 2, i+1), cmap='viridis', min=0.6, max=1.5)

plt.suptitle('PSF Maps Across the Sky', fontsize=16, y=0.95)
plt.tight_layout()
plt.show()

In [ ]:
# Plot airmass and sky brightness
fig = plt.figure(figsize=(16, 8))

# Airmass
plt.subplot(1, 2, 1)
hp.mollview(maps['g_airmass'], title='Airmass Map', 
            sub=(1, 2, 1), cmap='plasma', min=1.0, max=2.5)

# Sky brightness (g-band)
plt.subplot(1, 2, 2)
hp.mollview(maps['g_msky'], title='G-band Sky Brightness (mag/arcsec²)', 
            sub=(1, 2, 2), cmap='viridis_r', min=20, max=23)

plt.tight_layout()
plt.show()

In [ ]:
# Plot exposure time maps
fig = plt.figure(figsize=(16, 12))
for i, band in enumerate(['g', 'r', 'i', 'z']):
    plt.subplot(2, 2, i+1)
    hp.mollview(maps[f'{band}_tvis'], title=f'{band.upper()}-band Exposure Time (s)', 
                sub=(2, 2, i+1), cmap='viridis', min=30, max=150)

plt.suptitle('Exposure Time Maps', fontsize=16, y=0.95)
plt.tight_layout()
plt.show()

## 3. Sample Objects from the Sky

Now we'll sample objects uniformly across the sky and extract the observing conditions for each object:

In [ ]:
def sample_objects_from_sky(maps, nside, n_objects=5000):
    """Sample objects uniformly across the sky."""
    # Sample positions uniformly on the sphere
    theta = np.arccos(np.random.uniform(-1, 1, n_objects))
    phi = np.random.uniform(0, 2*np.pi, n_objects)
    
    # Convert to HEALPix pixels
    pixels = hp.ang2pix(nside, theta, phi)
    
    # Convert to RA, Dec
    ra = phi * 180/np.pi
    dec = (np.pi/2 - theta) * 180/np.pi
    
    # Sample true magnitudes
    # Make them correlated (typical galaxy colors)
    g_true = np.random.uniform(20, 25, n_objects)
    r_true = g_true - 0.5 + 0.3 * np.random.normal(0, 1, n_objects)
    i_true = r_true - 0.2 + 0.2 * np.random.normal(0, 1, n_objects)
    z_true = i_true - 0.1 + 0.2 * np.random.normal(0, 1, n_objects)
    
    # Create catalog
    catalog = pd.DataFrame({
        'ra': ra,
        'dec': dec,
        'pixel': pixels,
        'g': g_true,
        'r': r_true,
        'i': i_true,
        'z': z_true
    })
    
    # Extract observing conditions from maps
    for param_name, param_map in maps.items():
        catalog[param_name] = param_map[pixels]
    
    return catalog

# Sample objects
print("Sampling objects from the sky...")
catalog = sample_objects_from_sky(maps, nside, n_objects=5000)
print(f"Sampled {len(catalog)} objects")
print("\nFirst few objects:")
print(catalog[['ra', 'dec', 'g', 'r', 'i', 'z', 'g_psf', 'g_airmass', 'g_msky', 'g_tvis']].head())

## 4. Apply the Error Model

Now we'll apply the CustomErrorModel to get observed magnitudes with realistic errors:

In [ ]:
# Apply the custom error model
print("Applying CustomErrorModel with spatially varying parameters...")
err_model = CustomErrorModel()
catalog_with_errors = err_model(catalog, random_state=42)

print("\nError statistics:")
error_cols = ['g_err', 'r_err', 'i_err', 'z_err']
print(catalog_with_errors[error_cols].describe())

## 5. Create Maps of Results

Now let's create HEALPix maps showing the observed magnitudes and errors:

In [ ]:
def create_result_maps(catalog_with_errors, nside):
    """Create HEALPix maps from the catalog results."""
    npix = hp.nside2npix(nside)
    
    # Initialize maps with NaN
    result_maps = {}
    
    columns_to_map = ['g', 'r', 'i', 'z', 'g_err', 'r_err', 'i_err', 'z_err']
    
    for col in columns_to_map:
        result_maps[col] = np.full(npix, hp.UNSEEN)
    
    # Fill in the maps where we have data
    pixels = catalog_with_errors['pixel'].values
    
    for col in columns_to_map:
        # Average values for pixels with multiple objects
        for pixel in np.unique(pixels):
            mask = pixels == pixel
            if np.any(mask):
                result_maps[col][pixel] = np.mean(catalog_with_errors[col][mask])
    
    return result_maps

# Create result maps
print("Creating result maps...")
result_maps = create_result_maps(catalog_with_errors, nside)

## 6. Visualize the Results

Let's visualize the observed magnitudes and errors on the sky:

In [ ]:
# Plot observed magnitude maps
fig = plt.figure(figsize=(16, 12))
for i, band in enumerate(['g', 'r', 'i', 'z']):
    plt.subplot(2, 2, i+1)
    hp.mollview(result_maps[band], title=f'{band.upper()}-band Observed Magnitudes', 
                sub=(2, 2, i+1), cmap='viridis_r', min=20, max=25)

plt.suptitle('Observed Magnitude Maps', fontsize=16, y=0.95)
plt.tight_layout()
plt.show()

In [ ]:
# Plot photometric error maps
fig = plt.figure(figsize=(16, 12))
for i, band in enumerate(['g', 'r', 'i', 'z']):
    plt.subplot(2, 2, i+1)
    hp.mollview(result_maps[f'{band}_err'], title=f'{band.upper()}-band Photometric Errors', 
                sub=(2, 2, i+1), cmap='plasma', min=0, max=0.2)

plt.suptitle('Photometric Error Maps', fontsize=16, y=0.95)
plt.tight_layout()
plt.show()

## 7. Compare with Constant Conditions

Let's compare the results with what we'd get from constant observing conditions:

In [ ]:
# Create catalog with constant conditions (no per-object parameters)
catalog_const = catalog[['ra', 'dec', 'pixel', 'g', 'r', 'i', 'z']].copy()
catalog_const_errors = err_model(catalog_const, random_state=42)

print("Comparison of error distributions:")
print("\nVariable conditions:")
print(catalog_with_errors[error_cols].describe())
print("\nConstant conditions:")
print(catalog_const_errors[error_cols].describe())

# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for i, band in enumerate(['g', 'r', 'i', 'z']):
    err_col = f'{band}_err'
    
    axes[i].hist(catalog_const_errors[err_col], bins=50, alpha=0.7, 
                label='Constant conditions', density=True, color='blue')
    axes[i].hist(catalog_with_errors[err_col], bins=50, alpha=0.7, 
                label='Variable conditions', density=True, color='red')
    
    axes[i].set_xlabel(f'{band.upper()}-band error (mag)')
    axes[i].set_ylabel('Density')
    axes[i].set_title(f'{band.upper()}-band Error Distribution')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Analyze Correlations

Let's analyze how the observing conditions correlate with the photometric errors:

In [ ]:
# Calculate correlations
print("Correlation between observing conditions and G-band errors:")
obs_params = ['g_psf', 'g_airmass', 'g_msky', 'g_tvis']
correlations = {}

for param in obs_params:
    corr = np.corrcoef(catalog_with_errors[param], catalog_with_errors['g_err'])[0, 1]
    correlations[param] = corr
    print(f"  {param}: {corr:.3f}")

# Plot correlations
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

param_labels = ['PSF (arcsec)', 'Airmass', 'Sky brightness (mag/arcsec²)', 'Exposure time (s)']

for i, (param, label) in enumerate(zip(obs_params, param_labels)):
    # Create 2D histogram for cleaner visualization
    axes[i].hexbin(catalog_with_errors[param], catalog_with_errors['g_err'], 
                   gridsize=30, cmap='viridis', alpha=0.7)
    axes[i].set_xlabel(label)
    axes[i].set_ylabel('G-band error (mag)')
    axes[i].set_title(f'Error vs {label}\n(r = {correlations[param]:.3f})')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Show Error vs Magnitude Relationships

Finally, let's show how the errors vary with magnitude in different regions of the sky:

In [ ]:
# Divide sky into regions based on observing conditions
# Good conditions: low PSF, low airmass
good_conditions = ((catalog_with_errors['g_psf'] < 1.0) & 
                   (catalog_with_errors['g_airmass'] < 1.5))

# Poor conditions: high PSF, high airmass
poor_conditions = ((catalog_with_errors['g_psf'] > 1.2) & 
                   (catalog_with_errors['g_airmass'] > 1.8))

# Plot error vs magnitude for different conditions
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for i, band in enumerate(['g', 'r', 'i', 'z']):
    err_col = f'{band}_err'
    
    # Plot all points
    axes[i].scatter(catalog_with_errors[band], catalog_with_errors[err_col], 
                   alpha=0.3, s=1, color='gray', label='All')
    
    # Highlight good conditions
    if np.any(good_conditions):
        axes[i].scatter(catalog_with_errors[band][good_conditions], 
                       catalog_with_errors[err_col][good_conditions], 
                       alpha=0.7, s=2, color='green', label='Good conditions')
    
    # Highlight poor conditions
    if np.any(poor_conditions):
        axes[i].scatter(catalog_with_errors[band][poor_conditions], 
                       catalog_with_errors[err_col][poor_conditions], 
                       alpha=0.7, s=2, color='red', label='Poor conditions')
    
    axes[i].set_xlabel(f'{band.upper()} magnitude')
    axes[i].set_ylabel(f'{band.upper()} error (mag)')
    axes[i].set_title(f'{band.upper()}-band: Error vs Magnitude')
    axes[i].grid(True, alpha=0.3)
    axes[i].legend()
    axes[i].set_yscale('log')

plt.tight_layout()
plt.show()

print(f"Good conditions: {np.sum(good_conditions)} objects")
print(f"Poor conditions: {np.sum(poor_conditions)} objects")
print(f"Total objects: {len(catalog_with_errors)}")

## Summary

This notebook demonstrated:

1. **Creating realistic HEALPix maps** of observing conditions (PSF, airmass, sky brightness, exposure time)
2. **Sampling objects** uniformly across the sky and extracting per-object parameters
3. **Applying the CustomErrorModel** with spatially varying parameters
4. **Visualizing results** on HEALPix maps showing observed magnitudes and errors
5. **Comparing with constant conditions** to show the effect of spatial variation
6. **Analyzing correlations** between observing conditions and photometric errors

Key findings:
- Spatially varying conditions significantly broaden the error distribution
- PSF and airmass are positively correlated with errors (as expected)
- Sky brightness and exposure time show the expected negative correlation with errors
- Error patterns on the sky reflect the underlying observing condition maps

The CustomErrorModel successfully handles per-object parameters from HEALPix maps, making it suitable for realistic wide-field survey simulations.